In [2]:
import pandas as pd
import pickle

In [ ]:
dses = ['tess', 'ravdess', 'savee', 'crema', 'subesco', 'bser', 'all_en', 'all_bn', 'all']
folders = ['vggish', 'yamnet']

res = []

for ds in dses:
    for folder in folders:
        with open(f'./{folder}/{ds}.pkl', 'rb') as f:
            data = pickle.load(f)


        for datum in data:
            size, dropout, opt, _, lr, _, _ = datum['info']
            res.append(
                {
                    'dataset': ds,
                    'model': folder,
                    'size': ', '.join([str(s) for s in size]),
                    'dropout': dropout if dropout != 0.0 else 'None',
                    'opt': opt,
                    'lr': lr,
                    'accuracy': datum['accuracy'],
                    'f1_weighted': datum['f1_weighted'],
                }
            )

df = pd.DataFrame(res)

best_df = pd.DataFrame()
for ds in dses:
    for folder in folders:
        best = df[(df['dataset'] == ds) & (df['model'] == folder)].sort_values('f1_weighted', ascending=False).iloc[0]
        best_df = best_df.append(best)
best_df.to_csv('best.csv', index=False)
best_df